# `TemporalDense` Demo

This notebook demos the `TemporalDense` layer implemented in `temporal_dense.py`.

Suppose the input has the shape $(b, t, d)$ where $b$ is the batch dimension, $t$ is the time dimension (i.e.: number of objects), and $d$ is the object dimension.

**Hyperparamters:** number of temporal neurons/units $n$, activation function $\text{Activation}$, whether to use bias, (regularizers, initializers, constraints, etc.)

**Trainable parameters:** kernel $W \in \mathbb{R}^{t \times n}$, bias $b \in \mathbb{R}^n$

The layer performs the following operation:

$$
\begin{align}
    x &\gets \texttt{einsum(`tn,btd->nd'}, W, \texttt{inputs)}\\
    x &\gets x + b\\
    x &\gets \text{Activation}(x)
\end{align}
$$

For $v_k = \texttt{inputs[:, :, k]} \in \reals^t$, this is equivalent to 

$$
    v_k \gets \text{Activation}(W^\top v_k + b)
$$



In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import tensorflow as tf

## Dummy data

In [2]:
num_objects = 2
object_dim = 4
batch_dim = 1

# generate data where features are bernoulli(1/2) 
data = np.random.randint(0, 2, size=(batch_dim, num_objects, object_dim))
data.shape

(1, 2, 4)

## Default configuration (`units = None, activation = 'tanh', use_bias=False`)

In [3]:
# create TemporalDense layer
# setting units=None => # of temporal neurons = # of objects

from temporal_dense import TemporalDense
tdense = TemporalDense(units=None, activation='tanh', use_bias=False)

In [4]:
tdense(data)

<tf.Tensor: shape=(1, 2, 4), dtype=float32, numpy=
array([[[0.795749  , 0.17126891, 0.85103315, 0.795749  ],
        [0.27428737, 0.14592347, 0.4040392 , 0.27428737]]], dtype=float32)>

In [5]:
# now, set the TemporalDense layer's kernel to be the defulat one which normalizes by the temporal mean
default_kernel = 100*(np.identity(num_objects) - 1/num_objects * np.ones(shape=(num_objects, num_objects)))
tdense.kernel.assign(default_kernel);

In [6]:
# the transformed data is now in the range (-1, 1) as desired
tdense(data)

<tf.Tensor: shape=(1, 2, 4), dtype=float32, numpy=
array([[[ 1., -1.,  0.,  1.],
        [-1.,  1.,  0., -1.]]], dtype=float32)>

## A more general configuration

By setting the `units` argument, we can choose the number of "temporal neurons". This controls the number of "objects" in the output sequence.

We can also set the `activation` argument to be any activation function, not necessarily `'tanh'`.

Finally, we can add a `bias`.

In [7]:
tdense = TemporalDense(units=5, activation='relu', use_bias=True)
tdense(data)

<tf.Tensor: shape=(1, 5, 4), dtype=float32, numpy=
array([[[0.79003906, 0.00551224, 0.7955513 , 0.79003906],
        [0.        , 0.8442383 , 0.3984375 , 0.        ],
        [0.38208008, 0.47436523, 0.8564453 , 0.38208008],
        [0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.45507812, 0.        , 0.        ]]], dtype=float32)>

Note that the number of objects in the sequence was transformed from 2 to 5. The dimension of the objects is the same since processing is done along the temporal axis.